In [15]:
import math
import time
from dataclasses import dataclass
import matplotlib.pyplot as plt
import numpy as np

In [16]:
@dataclass
class Genetic:
    best_fitness: int
    average_fitness: float

    def __init__(self, best, average):
        self.best_fitness = best
        self.average_fitness = average

In [17]:
class Evolution():
    def __init__(self, max = 256, size=10, base = 8):
        self.max = max
        self.size = size
        self.base = base
        self.generations = []
        self.first_gen = np.random.randint(0, self.max, size=size)
        self.generations.append(self.first_gen)
        self.target = np.random.randint(0, self.max)

    def mutation(self, x: int):
        start = np.random.randint(0, math.floor(self.base/2))
        end = np.random.randint(start, math.floor(self.base))
        x_str = str(bin(x))[2:]

        if len(x_str) != self.base:
            for i in range(self.base-len(x_str)):
                x_str = '0' + x_str

        x_str = list(x_str)
        for i in range(start, end):
            x_str[i] = '1' if x_str[i] == '0' else '0'

        return int(''.join(x_str), 2)

    def crossover(self, x, y):
        rnd = np.random.randint(0, 2)
        x_str = str(bin(x if rnd == 0 else y))[2:]
        y_str = str(bin(x if rnd == 1 else y))[2:]
        n = 0

        if len(x_str) % 2 == 0:
            n = len(x_str) / 2
        else: 
            if self.fitness(x if rnd == 0 else y) > self.fitness(x if rnd == 1 else y):
                n = math.ceil(len(x_str) / 2)
            else:
                n = math.floor(len(x_str) / 2)

            n = int(n)
            child = x_str[:n]+y_str[n:]
            return int(child, 2)

    def fitness(self, x: int):
        return -np.abs(x - self.target)

    def get_new_crossover(self, gen: zip):
        new_gen = []
        t = 0
        best = 0
        second = 0

        for i in range(len(gen)):
            if best > gen[i][0] > second != gen[i][1]:
                second = gen[i][1]

            if gen[i][0] > best != gen[i][1]:
                second = best
                best = gen[i][1]
                
        for i in gen:
            new_gen.append(self.crossover(i[1], best))
            new_gen.append(self.crossover(i[1], second))

        return new_gen

    def get_new_gen(self, old_gen):
        new_gen = self.get_new_crossover(old_gen)

        for i in np.random.randint(0, self.size, size=math.floor(self.size/2)):
            new_gen[i] = self.mutation(new_gen[i])

        return new_gen

    def train(self):
        i = 0
        fs = []
        while True:

            fitnesses = [self.fitness(x) for x in self.generations[i]]
            fs.append(fitnesses)
            # Pick 5 best numbers
            gen = sorted(zip(fitnesses, self.generations[i]), reverse=True)[:(math.floor(self.size/2))]
            if max(fitnesses) == 0:
                print(self.generations)
                return [Genetic(max(f), sum(f)/len(f)) for f in fs], self.generations

            new_gen = self.get_new_gen(gen)
            #print(new_gen)
            #print(self.target)
            self.generations.append(new_gen)

            i += 1

    def train_time(self):
        i = 0
        sts = []

        for t in range(20):
            print(i)
            st = time.time()
            while True:

                fitnesses = [self.fitness(x) for x in self.generations[i]]
                # Pick 5 best numbers
                gen = sorted(zip(fitnesses, self.generations[i]), reverse=True)[:(math.floor(self.size/2))]
                if max(fitnesses) == 0:
                    et = time.time()
                    sts.append(et-st)
                    break
                #print(gen)
                new_gen = self.get_new_gen(gen)
                self.generations.append(new_gen)

                i += 1

        return sum(sts)/len(sts)




In [18]:
def print_genetics(genetic, best, average, i):
    print(f"Genetic {i}: ")
    print(genetic)
    print(f"Best fitness: {best}")
    print(f"Average fitness: {average}")
    print("")

In [19]:
def task1():
    evolution = Evolution()

    genetic, genetics = evolution.train()

    for i in range(len(genetics)):
        print_genetics(genetics[i], genetic[i].best_fitness, genetic[i].average_fitness, i)
    
    print(f"Target: {evolution.target}")

In [25]:
def task2():
    n = np.arange(8, 18)
    time = []

    for i in range(8, 18):
        evolution = Evolution(2**i, 20, i)
        time.append(evolution.train_time())
        print(f"Done with: {2**i}")
        print(f"Target: {evolution.target}")
    plt.xlabel("Bit length")
    plt.ylabel("Time (s)")
    plt.plot(n, time)
    plt.show()